# Data Processing U850/V850 all files
Use with conda environment: geocat_comp_env  
Teagan King, John Truesdale, Katie Dagon  
Updated Feb 2022  

## Import libraries

In [1]:
import glob
import xarray as xr
import cftime
from datetime import datetime
import geocat.comp as gc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

### Set up Dask

In [2]:
# Import dask
import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
nmem='50GB' # specify memory here so it duplicates below
cluster = PBSCluster(
    cores=1, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='/glade/scratch/$USER/local_dask', # Use your local directory
    resource_spec='select=1:ncpus=1:mem='+nmem, # Specify resources
    account='P93300313', # Input your project ID here, previously this was known as 'project', now is 'account'
    walltime='04:30:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(75)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/tking/.conda/envs/geocat_comp_env/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43830 instead
  warnings.warn(


In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/43830/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/43830/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.49:45962,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/43830/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
# Use this cell to stop the dask workers
# cluster.scale(0)

## Read in files and set pressure level(s)

In [5]:
# location of data
datadir = '/glade/scratch/tking/cgnet/rcp85_2086_2100/'

ufile_format = '*.U.*.nc' # U files are b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4, 3hourly avg from 2080-2100
vfile_format = '*.V.*.nc' # V files are b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4, 3hourly avg from 2080-2100
psfile_format = '*.PS.*.nc' # PS files are b.e13.BRCP85C5CN.ne120_g16.003a.cam.h3, 3hourly avg from 2080-2100

# desired pressure level:
plevel = 850.0  # hPa

# use glob.glob() to get actual files, and sort in order to retain correctly matching files!
ufilenames = sorted(glob.glob(datadir+ufile_format))
vfilenames = sorted(glob.glob(datadir+vfile_format))
psfilenames = sorted(glob.glob(datadir+psfile_format))

# make new .nc files but replace U/V with U850/V850
unew_files = []
for filename in ufilenames:
    unew_files.append((filename.replace('.U.', '.U850.')).split('/')[-1])

vnew_files = []
for filename in vfilenames:
    vnew_files.append((filename.replace('.V.', '.V850.')).split('/')[-1])

In [6]:
# # Use this cell if need to re-run notebook for subset of files, eg if connection breaks before all files are complete
# ufilenames = ufilenames[6:]
# vfilenames = vfilenames[6:]
# psfilenames = psfilenames[6:]
# unew_files = unew_files[6:]
# vnew_files = vnew_files[6:]

In [ ]:
# print(ufilenames)
# print(vfilenames)
# print(psfilenames)
# print(vnew_files)
# print(unew_files)

### Run all files
This can take a bit over an hour per file...

In [ ]:
for wind in ['U', 'V']:
    for file_index in range(len(psfilenames)):        
        if wind=='U':
            windfile = xr.open_dataset(ufilenames[file_index], chunks={"time": 100, "ncol":10000}) # adjusting chunks for ~100mb size chunks
            description = 'Zonal wind at 850 mbar pressure surface'
            windname='U850'
        elif wind=='V':
            windfile = xr.open_dataset(vfilenames[file_index], chunks={"time": 100, "ncol":10000}) # adjusting chunks for ~100mb size chunks
            description='Meridional wind at 850 mbar pressure surface'
            windname='V850'
        psfile = xr.open_dataset(psfilenames[file_index], chunks={"time": 100, "ncol":10000}) # using the same chunk size for consistency
        print('working on {}'.format(psfilenames[file_index].split('.PS.')[-1][:4]))
        
        # interpolate to get correct grid levels using GeoCAT's interpolate hybrid to pressure function:
        plevdata = gc.interpolation.interp_hybrid_to_pressure(windfile[wind],  # 3d field U/V (time x lev x ncol)
                            psfile['PS'],  # surface pressure (time x ncol)
                            windfile['hyam'], windfile['hybm'],  # coefficients to calculate pressure at each level
                            p0=psfile['P0'].values,  # reference pressure
                            new_levels=np.array([85000], dtype='float32'),  # interpolate to 850 pressure level
                            lev_dim=None,  # lev is default
                            method='log', # use log because pressure falls off logarithmically

                            extrapolate=True, # extrapolate below ground values
                            variable='other',
                            # define dummy values for t_bot and phi_sfc
                            # reasoning for this described in GitHub issue: https://github.com/NCAR/geocat-comp/issues/26
                            # Hopefully the need for this will be removed soon.
                            t_bot=xr.DataArray([]),  # xarray.DataArray Temperature in Kelvin at the lowest layer of the model.
                                             # Not necessarily the same as surface temperature.
                                             # Required if ``extrapolate`` is True.
                            phi_sfc=xr.DataArray([])  # Geopotential in J/kg at the lowest layer of the model.
                                             # Not necessarily the same as surface geopotential.
                                             # Required if ``extrapolate`` is True.
                            )   #TODO: implement dask here
        print('finished interpolation')
        
        plevdata = plevdata.squeeze(dim='plev', drop=True).assign_attrs({'long_name' : description,
                                          'units' : 'm/s',
                                          'title' : '3-hr averaged {}'.format(description),
                                          'summary' : 'Create input for ClimateNet application',
                                          'creator' : 'Teagan King, tking@ucar.edu',
                                          'contact' : 'Teagan King, tking@ucar.edu',
                                          'script' : 'jupyter notebook at ML-extremes/notebooks/Data_Processing_U850_V850.ipynb',
                                          'conventions' : 'CF 1.0',
                                          'creation_date' : datetime.today().strftime('%Y-%m-%d'),
                                         })

        plevdata.name=windname
        
        if wind == 'U':
            file_save = unew_files[file_index]
        elif wind == 'V':
            file_save = vnew_files[file_index]

        plevdata.to_netcdf('/glade/scratch/tking/cgnet/rcp85_2086_2100/{}'.format(file_save))
        print('generated {}'.format(file_save))

working on 2086
finished interpolation
generated b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2086010100Z-2086123121Z.nc
working on 2087
finished interpolation


In [ ]:
# check dashboard listed above for dask progress board

In [ ]:
ls /glade/scratch/tking/cgnet/rcp85_2086_2100/*U850*.nc

In [ ]:
# %%bash
# module load nco/4.7.9

# copy printed files above into U850_V850_to_regrid list below..

# RCP 8.5
U850_V850_to_regrid=['/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2080010100Z-2080123118Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2081010100Z-2081123118Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2086010100Z-2086123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2087010100Z-2087123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2088010100Z-2088123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2089010100Z-2089123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2090010100Z-2090123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2091010100Z-2091123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2092010100Z-2092123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2093010100Z-2093123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2094010100Z-2094123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2095010100Z-2095123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2096010100Z-2096123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2097010100Z-2097123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2098010100Z-2098123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2099010100Z-2099123121Z.nc',
'/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2100010100Z-2100123121Z.nc']

for file in U850_V850_to_regrid:
    print('ncremap -m /glade/campaign/cgd/amp/jet/ClimateNet/data_processing/maps/map_ne120_to_0.23x0.31_bilinear.nc -i {} -o /glade/scratch/tking/cgnet/rcp85_2086_2100/regridded_U850_V850/{}'.format(file, file))

# update batch script (batch_psl_remap.sh) in /scripts directory to use your email, project number, etc
# update relevant script (psl_remap_script) in /scripts directory to include correct filenames (eg, replace current script with output from this cell that use your filenames)
# run these commands from casper terminal with `qsub <batch_script_name>`